In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
a=time.time()

## 1. Obtención de cookies

En esta sección se obtienen las cookies de la página para que la consulta funcione.

In [4]:
url = "https://www.sbs.gob.pe/app/stats/tc-cv-historico.asp"
session = requests.Session() # Creamos una sesión para guardar las cookies
headers_get = { 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "es-ES,es;q=0.9",
    "Upgrade-Insecure-Requests": "1",
}
response = session.get(url, headers=headers_get) #Con el método get entramos a la página y guargamos las cookies
if response.status_code == 200:
    print("Cookies obtenidas exitosamente.")
else:
    print("Hubo un error")

Cookies obtenidas exitosamente.


## 2. Ejecución de consulta a la API

Aquí se procede con la consulta a la API, los elementos variables son los del diccionario 'data'

In [6]:
headers_post = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "es-ES,es;q=0.9",
    "Referer": url,
    "Origin": "https://www.sbs.gob.pe",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
}

inicio=str(input('Ingrese fecha de inicio en formato DD/MM/YYYY: '))
fin=str(input('Ingrese fecha de en formato DD/MM/YYYY: '))

# Filtros para ejecutar la consulta a la API
data = {
    "FECHA_CONSULTA_1": inicio,  # Fecha de inicio
    "FECHA_CONSULTA_2": fin,  # Fecha de fin
    "s_moneda": "02", # "02" es el código para el dólar
    "button22": "Consultar"
}

# Realizar la solicitud POST (se hace desde la sesión guardada)

response = session.post(url, headers=headers_post, data=data)
if response.status_code == 200:
    print("Consulta ejecutada.")
else:
    print("Hubo un error")

Ingrese fecha de inicio en formato DD/MM/YYYY:  01/01/2024
Ingrese fecha de en formato DD/MM/YYYY:  31/12/2024


Consulta ejecutada.


## 3. Obtención de la información

Se parsea la información obtenida y se extrae la tabla del html.

In [8]:
response_text=response.text
soup = BeautifulSoup(response_text, 'html.parser') # Parseo del código html
tabla = soup.find('table', class_='APLI_tabla')
filas = tabla.find_all('tr')
data = []
for fila in filas:
    celdas = fila.find_all(['td', 'th'])
    data.append([celda.get_text(strip=True) for celda in celdas])

df = pd.DataFrame(data[2:], columns=data[1])

In [9]:
df

,FECHA,COMPRA(S/.),VENTA(S/.)
0,03/01/2024,3.726,3.738
1,04/01/2024,3.736,3.740
2,05/01/2024,3.713,3.723
3,08/01/2024,3.710,3.719
4,09/01/2024,3.703,3.708
...,...,...,...
241,23/12/2024,3.721,3.735
242,24/12/2024,3.718,3.731
243,26/12/2024,3.738,3.748
244,27/12/2024,3.741,3.750


## 4. (Opcional) Complementos a la tabla

En esta sección se rellenan fechas sin información (días no hábiles)

In [11]:
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d/%m/%Y', errors='coerce')

In [12]:
inicio=pd.to_datetime(inicio).date()
fin=pd.to_datetime(fin).date()
rango_fechas = pd.date_range(start=inicio, end=fin)

C:\Users\User\AppData\Local\Temp\ipykernel_6988\1578155632.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  fin=pd.to_datetime(fin).date()


In [13]:
df = df.set_index('FECHA').reindex(rango_fechas)
df.index.name = 'FECHA'
for i in df.columns.tolist():
    '''
    El método ffill llena valores hacia adelante
    Por ejemplo: si se tiene un dato del 27/12 y luego no hay hasta el 31/12, los días intermedios tendrán la información del 27/12)
    '''
    df[i]=df[i].ffill() 
    '''
    El método bfill llena valores hacia atrás
    En este caso no hay información del 1 y 2 de enero, este método hace que se rellene dicha información con el valor más cercano (3 de enero)
    '''
    df[i]=df[i].bfill()

In [14]:
df

,COMPRA(S/.),VENTA(S/.)
FECHA,,
2024-01-01,3.726,3.738
2024-01-02,3.726,3.738
2024-01-03,3.726,3.738
2024-01-04,3.736,3.740
2024-01-05,3.713,3.723
...,...,...
2024-12-27,3.741,3.750
2024-12-28,3.741,3.750
2024-12-29,3.741,3.750


In [15]:
b=time.time()
print(f'Proceso terminado en {round((b-a),0)} segundos')

Proceso terminado en 11.0 segundos
